In [ ]:
import os
import shap
import json
import time

import pandas as pd
import seaborn as sns
import numpy as np

from matplotlib import pyplot as plt
from sklearn.model_selection import RepeatedKFold, LeaveOneOut, GridSearchCV
from sklearn import neighbors, metrics
from dotenv import load_dotenv

from models.linear_regressions import Linear_reg
from scripts.model_actions import freeze_model

load_dotenv()
sns.color_palette('colorblind')
plt.style.use('Solarize_Light2')

# Setting default DPI, pulling it from dotenv if it exists, setting it on 100 if not

pc_dpi = int(os.getenv('DPI'))

if pc_dpi is None:
    pc_dpi = 100


# <u>Tentative de modélisation et prédiction de la variable : Intensité d'émission de GàES</u>

## <u>1 : Modélisations en prenant en compte la note Energy Star (E*)</u>
### <u>1.1 : Régressions linéaires</u>

## <u>2 : Étude de l'importance de la note Energy Star</u>
### <u>2.1 : Feature importance via SHAP </u>
### <u>2.2 : Modélisation sans utiliser de variables E*</u>

## <u>3 : Application d'une méthode non-linéaire</u>

<hr>

## <u>1 : Modélisations prenant en compte toutes les variables retenues lors de l'étude</u>
### <u>1.1 : Régressions linéaires</u>

#### 1.1.a : Cross validation = Leave One Out
- On utilise dans un premier temps toutes les variables retenues lors de l'analyse exploratoire.
- Un split satisfaisant à déjà été trouvé et fixé, ces données viendront sur-écrire le split proposé par le modèle (ici la Classe) par défaut
- On effectue 4 régressions (OLS, Ridge, Lasso et Elastic Net) avec les paramètres par défaut de la classe pour la validation croisée : Leave One Out

In [ ]:
general_file = "./data/seattle_std_scaled.csv"  # Used as backup


In [ ]:
df_ghg = pd.read_csv(general_file)
df_ghg.set_index("OSEBuildingID", inplace=True)


In [ ]:
df_ghg.head()


In [ ]:
df_ghg.columns


In [ ]:
# Target : target_GHGEmissionsIntensity(kgCO2e/ft2) :

droplist = [
    "scaled_GHGEmissionsIntensity(kgCO2e/ft2)",  # Scaled target
    "target_SourceEUI(kWh/m2)",  # not to scale
    "EnergyStarCert",
    ]

df_model = df_ghg.drop(columns=droplist)


In [ ]:
ghg_target = "target_GHGEmissionsIntensity(kgCO2e/ft2)"
ghg_regression = Linear_reg(dataframe=df_model, target=ghg_target)


In [ ]:
# Loading known split, ids are unique building OSE id

with open("./data/splits_ghg.json", "r") as json_file:
    splits = json.load(json_file)

ids_train = splits["train"]
ids_test = splits["test"]


In [ ]:
# Overriding

df_train_override = df_model[df_model.index.isin(ids_train)]
df_test_override = df_model[df_model.index.isin(ids_test)]

ghg_regression.force_split(
    df_train_ovr=df_train_override,
    df_test_ovr=df_test_override
)


#### Exécution : 
Paramètres : 
- Ridge = 0.1, 45, step 0.05
- Elastic Net = Alpha = Alpha_ridge = 0.1, 45, step 0.05, default L1 ratio
- Lasso Alpha = 0.01, 15, 0.04

In [ ]:
# Recheck default : 
ghg_regression.common_parameters["cv"] = None
#

alphas_ridge = np.arange(0.1, 45, 0.05)
alphas_elnet = alphas_ridge
alphas_lasso = np.arange(0.01, 15, 0.04)

ghg_regression.execute_all(
    alphas_elnet=alphas_elnet,
    alphas_ridge=alphas_ridge,
    alphas_lasso=alphas_lasso
)



##### --> 1 Visualisation des alphas et leur pertinence, affichage des meilleurs hyperparamètre
##### --> 2 Comparaison des métriques

In [ ]:
print("Elnet : \n")
print(f"Elastic Net L1 Ratio : {ghg_regression.elnet_cv.l1_ratio_}")
print(f"Elastic Net best Alpha : {ghg_regression.elnet_cv.alpha_}")
print(f"Time used during fit = {ghg_regression.elnet_time_card.t_fit}")
print(f"Time used during predict = {ghg_regression.elnet_time_card.t_predict}")


In [ ]:
print("Ridge: \n")
print(f"Ridge best Alpha : {ghg_regression.ridge_cv.alpha_}")
print(f"Time used during fit = {ghg_regression.ridge_time_card.t_fit}")
print(f"Time used during predict = {ghg_regression.ridge_time_card.t_predict}")

ghg_regression.ridge_plot()


In [ ]:
print("LASSO: \n")
print(f"LASSO best Alpha : {ghg_regression.lasso_cv.alpha_}")
print(f"Time used during fit = {ghg_regression.lasso_time_card.t_fit}")
print(f"Time used during predict = {ghg_regression.lasso_time_card.t_predict}")

ghg_regression.lasso_plot()


In [ ]:
metrics_with_estar_l1out = ghg_regression.format_all_metrics()

df_predictions_estar_l1out = ghg_regression.df_predictions

metrics_with_estar_l1out


In [ ]:
# Saving Ridge Model and its predictions : 

ridge_l1out_estar = ghg_regression.ridge_cv
predictions_ridge_l1out_estar = ghg_regression.df_predictions["Ridge"]


##### Observation 1.1.a : 
- Ridge semble être le plus prometteur dans ce cas avec un score R2 supérieur à 0.6 et un rapport test/test stable.
- L'erreur est de .695 et .686 kg de CO2/m2 pour les données train/test, respectivement. Le plus bas de toutes les régressions
- On suit particulièrement la régression Ridge lors du passage a la vérification via Kfolds

In [ ]:
# freeze_model(model=ghg_regression, save_file=True, file_path="./data/ghg_splits.json")
# uncomment to save train test split ids (OSEBuildingID)


#### 1.1.b : Cross validation = RepeatedKFold
- Mêmes paramètres que 1.1.a
- On effectue 4 régressions (OLS, Ridge, Lasso et Elastic Net), mise à jour des paramètres par défaut de la classe pour la validation croisée : RepeatedKfold(30 fois : 10 splits, 3 exécutions)
- On attend des temps de traitement nettement supérieurs

In [ ]:
k_folds = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
ghg_regression.common_parameters["cv"] = k_folds


In [ ]:
# Re executing all with adjusted cross validation parameters :

# alphas_ridge = np.arange(0.1, 45, 0.05)  <-- already assigned, commented as a reminder
# alphas_elnet = alphas_ridge
# alphas_lasso = np.arange(0.01, 15, 0.04)

ghg_regression.execute_all(
    alphas_elnet=alphas_elnet,
    alphas_ridge=alphas_ridge,
    alphas_lasso=alphas_lasso
)



In [ ]:
print("Elnet : \n")
print(f"Elastic Net L1 Ratio : {ghg_regression.elnet_cv.l1_ratio_}")
print(f"Elastic Net best Alpha : {ghg_regression.elnet_cv.alpha_}")
print(f"Time used during fit = {ghg_regression.elnet_time_card.t_fit}")
print(f"Time used during predict = {ghg_regression.elnet_time_card.t_predict}")


In [ ]:
print("Ridge: \n")
print(f"Ridge best Alpha : {ghg_regression.ridge_cv.alpha_}")
print(f"Time used during fit = {ghg_regression.ridge_time_card.t_fit}")
print(f"Time used during predict = {ghg_regression.ridge_time_card.t_predict}")


In [ ]:
print("LASSO: \n")
print(f"LASSO best Alpha : {ghg_regression.lasso_cv.alpha_}")
print(f"Time used during fit = {ghg_regression.lasso_time_card.t_fit}")
print(f"Time used during predict = {ghg_regression.lasso_time_card.t_predict}")


In [ ]:
metrics_with_estar_rkfold = ghg_regression.format_all_metrics()

df_predictions_estar_rkfold = ghg_regression.df_predictions

print("Kfold (10*3) : \n")

metrics_with_estar_rkfold


In [ ]:
print("Leave One Out : \n")

metrics_with_estar_l1out


##### Observations :
- Étonnamment, la validation croisée avec les paramètres utilisés (10 folds répétés 3 fois) n'offre pas de gain dans le cas de Ridge, au contraire.
- Les autres modèles de régression sont toujours moins performants, que ce soit au niveau de l'erreur ou du score R2, ainsi que la transition train/test
- Ridge Leave One Out se montre, dans ce cas, le meilleur candidat pour modéliser l'intensité d'émissions de GàES
- On utilise Ridge avec Leave One Out comme baseline.
- Le temps de traitement en utilisant la méthode Leave One Out est également beaucoup plus rapide que RKfolds

In [ ]:
# Reverting to cv = None for l1out

ghg_regression.common_parameters["cv"] = None


In [ ]:
ghg_regression.use_ridge_cv(alphas=alphas_ridge)  # obj.df_predictions actualized and reset for shap

ghg_regression.scatter_true_pred(regression_name="Ridge")


#### Conclusion, partie 1 : 

- Ridge semble être le modèle à la fois le plus précis et le plus consistant. La validation via Kfolds n'offre pas d'avantages mais des pertes, que ce soit en terme de précision ou de temps.

## <u>2 : Etude de l'importance de la note Energy Star</u>

### <u>2.1 : Feature importance via SHAP</u>

In [ ]:
X_all = ghg_regression.df_origin.drop(columns=[ghg_target]).to_numpy()
X100 = shap.utils.sample(X_all, 100) # 100 instances for use as the background distribution

features = ghg_regression.df_origin.drop(columns=[ghg_target]).columns

explainer = shap.LinearExplainer(ghg_regression.ridge_cv, X100, feature_names=features)
shap_values = explainer(X_all)


In [ ]:
sample_ind = 18
my_waterfall = shap.plots.waterfall(shap_values[0], max_display=33, show=False)
my_waterfall.figure.set_size_inches(10, 10)
my_waterfall.figure.set_dpi(pc_dpi)

my_waterfall.suptitle("Impact des differentes variables sur le modele, classées par ordre d'importance")

plt.show()


In [ ]:
summary = shap.plots.beeswarm(shap_values, max_display=32, show=False)

summary.figure.set_size_inches(15, 8)
summary.figure.set_dpi(pc_dpi)

summary.suptitle("Impact des differentes variables sur le modele, classées par ordre d'importance")

plt.show()


#### Observations :

Dans le cas de la modélisation de l'intensité d'émission de GàES, l'impact du score ENERGYSTAR ne semble pas avoir d'importance. En effet, les graphes ci dessus montrent que la variable influe extrêmement peu sur la sortie du modèle : <b>Waterfall n1 : index = 33, < .01 || Summary n2 : index = 32, impact extrêmement faible</b> <br>
Ici, on peut émettre l'hypothèse que ne pas utiliser E* n'appauvrirait que très peu nos modèles <br>
Le modèle sélectionné à l'issue de la première partie de cette analyse était Ridge en utilisant un système de validation Leave One Out. <br>
On peut adapter la classe pour qu'elle abandonne le score E* et voir comment cela se traduit au niveau du modèle.

### <u>2.2 : Modélisation sans utiliser de variables E*</u>

In [ ]:
drop_estar = ["ENERGYSTARScore"]
ghg_regression.drop_col(col_list=drop_estar)


In [ ]:
# checking ENERGYSTARScore not in matrices : shape should be 1 less than original dfs

if ghg_regression.X_test.shape[1] >= len(ghg_regression.df_train.columns) + 1:  # +1 because target still in cols
    print("Oops")
else:
    print("Ok")


#### 2.2.a : Cross validation = Leave One Out
- Energy Star Score ne figure plus dans les matrices X_train, X_test
- Mêmes paramètres initiaux que la régression comportant E*

#### Exécution :
Paramètres : 
- Ridge = 0.1, 45, step 0.05
- Elastic Net = Alpha = Alpha_ridge = 0.1, 45, step 0.05, default L1 ratio
- Lasso Alpha = 0.01, 15, 0.04

In [ ]:
ghg_regression.execute_all(
    alphas_elnet=alphas_elnet,
    alphas_lasso=alphas_lasso,
    alphas_ridge=alphas_ridge
    )


In [ ]:
print("Elnet : \n")
print(f"Elastic Net L1 Ratio : {ghg_regression.elnet_cv.l1_ratio_}")
print(f"Elastic Net best Alpha : {ghg_regression.elnet_cv.alpha_}")
print(f"Time used during fit = {ghg_regression.elnet_time_card.t_fit}")
print(f"Time used during predict = {ghg_regression.elnet_time_card.t_predict}")


In [ ]:
print("Ridge: \n")
print(f"Ridge best Alpha : {ghg_regression.ridge_cv.alpha_}")
print(f"Time used during fit = {ghg_regression.ridge_time_card.t_fit}")
print(f"Time used during predict = {ghg_regression.ridge_time_card.t_predict}")

ghg_regression.ridge_plot()


In [ ]:
print("LASSO: \n")
print(f"LASSO best Alpha : {ghg_regression.lasso_cv.alpha_}")
print(f"Time used during fit = {ghg_regression.lasso_time_card.t_fit}")
print(f"Time used during predict = {ghg_regression.lasso_time_card.t_predict}")

ghg_regression.lasso_plot()


In [ ]:
metrics_without_estar_l1out = ghg_regression.format_all_metrics()

df_predictions_no_estar_l1out = ghg_regression.df_predictions

metrics_without_estar_l1out


#### 2.2.b : Cross validation = RepeatedKfold
- Mêmes paramètres que 2.2.a
- On effectue 4 régressions (OLS, Ridge, Lasso et Elastic Net) mise à jour des paramètres par défaut de la classe pour la validation croisée : RepeatedKfold(30 fois : 10 splits, 3 exécutions)
- On attend des temps de traitement nettement supérieurs

In [ ]:
ghg_regression.common_parameters["cv"] = k_folds


In [ ]:
ghg_regression.execute_all(
    alphas_elnet=alphas_elnet,
    alphas_lasso=alphas_lasso,
    alphas_ridge=alphas_ridge
    )


In [ ]:
print("Elnet : \n")
print(f"Elastic Net L1 Ratio : {ghg_regression.elnet_cv.l1_ratio_}")
print(f"Elastic Net best Alpha : {ghg_regression.elnet_cv.alpha_}")
print(f"Time used during fit = {ghg_regression.elnet_time_card.t_fit}")
print(f"Time used during predict = {ghg_regression.elnet_time_card.t_predict}")


In [ ]:
print("Ridge: \n")
print(f"Ridge best Alpha : {ghg_regression.ridge_cv.alpha_}")
print(f"Time used during fit = {ghg_regression.ridge_time_card.t_fit}")
print(f"Time used during predict = {ghg_regression.ridge_time_card.t_predict}")


In [ ]:
print("LASSO: \n")
print(f"LASSO best Alpha : {ghg_regression.lasso_cv.alpha_}")
print(f"Time used during fit = {ghg_regression.lasso_time_card.t_fit}")
print(f"Time used during predict = {ghg_regression.lasso_time_card.t_predict}")


In [ ]:
print("Metrics using Leave One Out method, E* in model : \n")

metrics_with_estar_l1out


In [ ]:
metrics_without_estar_kfold = ghg_regression.format_all_metrics()

print("Metrics using repeated Kfold method (10, 3 repeats), E* not in model : \n")

metrics_without_estar_kfold


In [ ]:
print("Metrics using Leave One Out method, E* in model : \n")

metrics_with_estar_l1out


## <u>3 : Utilisation d'une méthode non linéaire : régression via KNN</u>

- Le problème semble être adapté à l'utilisation de méthodes de régressions linéaires
- On peut vérifier cela en utilisant une méthode de modélisation non linéaire (ici régression par KNN) et comparer les performances des deux modèles (comparaison avec le meilleur modèle linéaire : Ridge).
- On utilise exactement les mêmes données (provenant de la Classe Linear_reg), en gardant le même split.
- On effectue deux validations croisées : Une fois en utilisant LeaveOneOut, et une autre fois en utilisant Kfolds (mêmes paramètres que pour les régressions linéaires)
- On utilise la même métrique (MSE) pour la recherche d'hyperparamètre. 

In [ ]:
ghg_regression.reset_cols()  # We can reset the columns so that E* score is used

X_train, X_test = ghg_regression.X_train, ghg_regression.X_test
y_train, y_test = ghg_regression.y_train, ghg_regression.y_test

neighbors_candidates = [5, 7, 9, 11, 13, 15, 17, 19]

knnr = neighbors.KNeighborsRegressor()

# Grid search

l1out = LeaveOneOut()

param_grid_knnr = {'n_neighbors':neighbors_candidates}

score = "neg_mean_squared_error"

# KNN regressors Setup

knn_reg_l1o = GridSearchCV(
    estimator=knnr,
    param_grid=param_grid_knnr,
    cv=l1out,
    scoring=score
)

knn_reg_rkf = GridSearchCV(
    estimator=knnr,
    param_grid=param_grid_knnr,
    cv=k_folds,
    scoring=score
)


In [ ]:
knnr_l1o_fit_ts = time.perf_counter()

knn_reg_l1o.fit(
    X=X_train,
    y=y_train
)

knnr_l1o_fit_tf = time.perf_counter()

time_to_fit_knn_l1out = knnr_l1o_fit_tf - knnr_l1o_fit_ts

rmse_knnr_l1o = np.sqrt(abs(knn_reg_l1o.best_score_))
y_pred_train_l1o = knn_reg_l1o.predict(X_train)
r2_knnr_l1o = metrics.r2_score(y_true=ghg_regression.y_train, y_pred=y_pred_train_l1o)


In [ ]:
print(f"RMSE Regression KNN, Cross Val : Leave one out = {rmse_knnr_l1o}")
print(f"Score R2 Regression KNN, Cross Val : Leave one out = {r2_knnr_l1o}")
print(f"Temps fit : {time_to_fit_knn_l1out}")
print(f"Meilleur hyperparametre : {knn_reg_l1o.best_params_}")


#### Observations, cas régression KNN, validation croisée Repeated Kfolds : 
- Le score R2 est beaucoup plus faible tous les autres scores obtenus lors de régressions linéaires
- L'erreur augmente également significativement
- Le temps de traitement pour fit (3.9s) est largement supérieur à celui de Ridge utilisant Leave One Out (0.071s)
- On effectue la même expérimentation en utilisant la validation croisée plus rigoureuse : KFolds répétés (3 * 10 folds) pour avoir une conclusion définitive sur le modèle de régression par KNN

In [ ]:
knnr_rkf_fit_ts = time.perf_counter()

knn_reg_rkf.fit(
    X=X_train,
    y=y_train
)

knnr_rkf_fit_tf = time.perf_counter()

time_to_fit_knn_rkf = knnr_rkf_fit_tf - knnr_rkf_fit_ts

rmse_knnr_rkf = np.sqrt(abs(knn_reg_rkf.best_score_))
y_pred_train_rkf = knn_reg_rkf.predict(X_train)
r2_knnr_rkf = metrics.r2_score(y_true=ghg_regression.y_train, y_pred=y_pred_train_l1o)


In [ ]:
print(f"RMSE Regression KNN, Cross Val : Repeated KFolds = {rmse_knnr_rkf}")
print(f"Score R2 Regression KNN, Cross Val : Repeated KFolds = {r2_knnr_rkf}")
print(f"Temps fit : {time_to_fit_knn_rkf}")
print(f"Meilleur hyperparametre : {knn_reg_rkf.best_params_}")


## Conclusions :

- Sur les modèles testés, le problème semble être plus aisément solvable en utilisant des méthodes de régressions linéaires (scores faibles en utilisant régression via KNN par ex.)

L'hypothèse selon laquelle E* serait indispensable à la modélisation de prédictions <u>concernant les émissions de GàES</u> semble fausse. Le modèle Ridge est suffisamment robuste. L'analyse via SHAP et les comparaisons entre les différentes méthodes nous montrent que, vis à vis de nos données, l'utilisation d'une régression Ridge est optimale. La note E* apporte trop peu au modèle pour être considérée comme vitale. Les gains apportés ne valent pas les efforts demandés pour l'obtenir.

- Recommandations issues de l'étude : Ridge utilisant comme hyperparamètre : 6.350000000000001
- On pourrait étendre ce modèle pour améliorer sa précision en utilisant d'autres features catégorielles (Matériaux de construction, entreprise contractée pour les travaux, date de modernisation etc.)